# 1 - Business Problem

In [1]:
import pandas as pd

df = pd.read_csv("./data/training-mltable-folder/bank_marketing_train_data.csv")
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,57,technician,married,high.school,no,no,yes,cellular,may,mon,...,1,999,1,failure,-1.8,92.893,-46.2,1.299,5099.1,no
1,55,unknown,married,unknown,unknown,yes,no,telephone,may,thu,...,2,999,0,nonexistent,1.1,93.994,-36.4,4.860,5191.0,no
2,33,blue-collar,married,basic.9y,no,no,no,cellular,may,fri,...,1,999,1,failure,-1.8,92.893,-46.2,1.313,5099.1,no
3,36,admin.,married,high.school,no,no,no,telephone,jun,fri,...,4,999,0,nonexistent,1.4,94.465,-41.8,4.967,5228.1,no
4,27,housemaid,married,high.school,no,yes,no,cellular,jul,fri,...,2,999,0,nonexistent,1.4,93.918,-42.7,4.963,5228.1,no


In [2]:
df.shape

(29999, 21)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29999 entries, 0 to 29998
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             29999 non-null  int64  
 1   job             29999 non-null  object 
 2   marital         29999 non-null  object 
 3   education       29999 non-null  object 
 4   default         29999 non-null  object 
 5   housing         29999 non-null  object 
 6   loan            29999 non-null  object 
 7   contact         29999 non-null  object 
 8   month           29999 non-null  object 
 9   day_of_week     29999 non-null  object 
 10  duration        29999 non-null  int64  
 11  campaign        29999 non-null  int64  
 12  pdays           29999 non-null  int64  
 13  previous        29999 non-null  int64  
 14  poutcome        29999 non-null  object 
 15  emp.var.rate    29999 non-null  float64
 16  cons.price.idx  29999 non-null  float64
 17  cons.conf.idx   29999 non-null 

In [4]:
df.describe()

,age,duration,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed
count,29999.000000,29999.000000,29999.000000,29999.000000,29999.000000,29999.000000,29999.000000,29999.000000,29999.000000,29999.000000
mean,40.042968,258.482849,2.567719,962.059369,0.174539,0.078083,93.575004,-40.513184,3.618437,5166.945855
std,10.443217,260.242432,2.783605,187.933424,0.497949,1.571887,0.578637,4.620560,1.734565,72.169284
min,17.000000,0.000000,1.000000,0.000000,0.000000,-3.400000,92.201000,-50.800000,0.634000,4963.600000
25%,32.000000,102.000000,1.000000,999.000000,0.000000,-1.800000,93.075000,-42.700000,1.344000,5099.100000
50%,38.000000,179.000000,2.000000,999.000000,0.000000,1.100000,93.749000,-41.800000,4.857000,5191.000000
75%,47.000000,319.000000,3.000000,999.000000,0.000000,1.400000,93.994000,-36.400000,4.961000,5228.100000
max,98.000000,4918.000000,56.000000,999.000000,7.000000,1.400000,94.767000,-26.900000,5.045000,5228.100000


# 2 - Connect to Azure Machine Learning Workspace

In [5]:
# Import required libraries
import os
from azure.identity import DefaultAzureCredential
from azure.ai.ml import automl, Input, MLClient, command

from azure.ai.ml.constants import AssetTypes
from azure.ai.ml.entities import Data
from azure.ai.ml.automl import (classification, ClassificationPrimaryMetrics, ClassificationModels)

In [6]:
# Configure workspace details and get a handle to workspace

# Details of azure ML workspace
subscription_id = "3b57d2fe-08b1-4fe9-b535-f5c4387b9a66"
resource_group = "automl-rg"
workspace = "automl-ws"

# Get a handle to workspace
ml_client = MLClient(DefaultAzureCredential(), subscription_id, resource_group, workspace)

In [7]:
# Show Azure ML Workspace information

workspace = ml_client.workspaces.get(name=ml_client.workspace_name)

subscription_id = ml_client.connections._subscription_id
resource_group = workspace.resource_group
workspace_name = ml_client.workspace_name

output = {}
output["Workspace"] = workspace_name
output["Subscription ID"] = subscription_id
output["Resource Group"] = resource_group
output["Location"] = workspace.location
output

{'Workspace': 'automl-ws',
 'Subscription ID': '3b57d2fe-08b1-4fe9-b535-f5c4387b9a66',
 'Resource Group': 'automl-rg',
 'Location': 'westus'}

# 3 - MLTable with input Training Data

In [8]:
# Create MLTable

import mltable

paths = [{'file': './data/training-mltable-folder/bank_marketing_train_data.csv'}]

train_table = mltable.from_delimited_files(paths)
train_table.save("./data/training-mltable-folder")

In [9]:
# Create MLTable data input

my_training_data_input = Input(type=AssetTypes.MLTABLE, path='./data/training-mltable-folder')

# 4 - Compute target setup

In [10]:
# Create or Attach existing AmlCompute

from azure.ai.ml.entities import AmlCompute
from azure.core.exceptions import ResourceNotFoundError

compute_name = "cpu-cluster"

try:
    _ = ml_client.compute.get(compute_name)
    print("Found existing compute target.")
except ResourceNotFoundError:
    print("Creating a new compute target...")
    compute_config = AmlCompute(
        name=compute_name,
        type="amlcompute",
        size="STANDARD_DS12_V2",
        idle_time_before_scale_down=120,
        min_instances=0,
        max_instances=6,
    )
    ml_client.begin_create_or_update(compute_config).result()

Creating a new compute target...


# 5 - Configure and run AutoML classification job

In [11]:
max_trials = 5
exp_name = "bank-marketing-classification-experiment"

In [12]:
# AutoML classification job
classification_job = automl.classification(
    compute=compute_name,
    experiment_name=exp_name,
    training_data=my_training_data_input,
    target_column_name='y',
    primary_metric='accuracy',
    n_cross_validations=5,
    enable_model_explainability=True,
    tags={"custom tag":"custom value"}
)

# Limits
classification_job.set_limits(
    timeout_minutes=60,
    trial_timeout_minutes=5,
    max_trials=max_trials,
    enable_early_termination=True
)

# Training properties (optional)
classification_job.set_training(
    blocked_training_algorithms=[ClassificationModels.LOGISTIC_REGRESSION],
    enable_onnx_compatible_models=True
)

In [13]:
# Submit AutoML job

returned_job = ml_client.create_or_update(classification_job)
print(f"Created job: {returned_job}")

Created job: compute: azureml:cpu-cluster
creation_context:
  created_at: '2024-04-27T04:10:05.867246+00:00'
  created_by: Vijay Gadhave
  created_by_type: User
display_name: dynamic_frog_vww91ltjw6
experiment_name: bank-marketing-classification-experiment
id: azureml:/subscriptions/3b57d2fe-08b1-4fe9-b535-f5c4387b9a66/resourceGroups/automl-rg/providers/Microsoft.MachineLearningServices/workspaces/automl-ws/jobs/dynamic_frog_vww91ltjw6
limits:
  enable_early_termination: true
  max_concurrent_trials: 1
  max_cores_per_trial: -1
  max_nodes: 1
  max_trials: 5
  timeout_minutes: 60
  trial_timeout_minutes: 5
log_verbosity: info
n_cross_validations: 5
name: dynamic_frog_vww91ltjw6
outputs: {}
primary_metric: accuracy
properties: {}
resources:
  instance_count: 1
  shm_size: 2g
services:
  Studio:
    endpoint: https://ml.azure.com/runs/dynamic_frog_vww91ltjw6?wsid=/subscriptions/3b57d2fe-08b1-4fe9-b535-f5c4387b9a66/resourcegroups/automl-rg/workspaces/automl-ws&tid=843e1e65-e875-4fbd-8ddc-

In [14]:
# Wait until the AutoML job is finished

ml_client.jobs.stream(returned_job.name)

RunId: dynamic_frog_vww91ltjw6
Web View: https://ml.azure.com/runs/dynamic_frog_vww91ltjw6?wsid=/subscriptions/3b57d2fe-08b1-4fe9-b535-f5c4387b9a66/resourcegroups/automl-rg/workspaces/automl-ws

Execution Summary
RunId: dynamic_frog_vww91ltjw6
Web View: https://ml.azure.com/runs/dynamic_frog_vww91ltjw6?wsid=/subscriptions/3b57d2fe-08b1-4fe9-b535-f5c4387b9a66/resourcegroups/automl-rg/workspaces/automl-ws



In [15]:
# URL for the status of job

returned_job.services["Studio"].endpoint

'https://ml.azure.com/runs/dynamic_frog_vww91ltjw6?wsid=/subscriptions/3b57d2fe-08b1-4fe9-b535-f5c4387b9a66/resourcegroups/automl-rg/workspaces/automl-ws&tid=843e1e65-e875-4fbd-8ddc-ce696f3a551d'

In [16]:
print(returned_job.name)

dynamic_frog_vww91ltjw6


# 6 - Retrieve Best Trial

### Obtain the tracking URI for MLFlow

In [17]:
# Obtain the tracking URI from MLClient

import mlflow

MLFLOW_TRACKING_URI = ml_client.workspaces.get(name=ml_client.workspace_name).mlflow_tracking_uri
print(MLFLOW_TRACKING_URI)

azureml://westus.api.azureml.ms/mlflow/v1.0/subscriptions/3b57d2fe-08b1-4fe9-b535-f5c4387b9a66/resourceGroups/automl-rg/providers/Microsoft.MachineLearningServices/workspaces/automl-ws


In [18]:
# Set the MLFLOW TRACKING URI

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
print("\nCurrent tracking uri: {}".format(mlflow.get_tracking_uri()))


Current tracking uri: azureml://westus.api.azureml.ms/mlflow/v1.0/subscriptions/3b57d2fe-08b1-4fe9-b535-f5c4387b9a66/resourceGroups/automl-rg/providers/Microsoft.MachineLearningServices/workspaces/automl-ws


In [19]:
# Initialize MLFlow client

from mlflow.tracking.client import MlflowClient
from mlflow.artifacts import download_artifacts

mlflow_client = MlflowClient()

### Get the AutoML parent Job

In [20]:
job_name = returned_job.name

# Get the parent run
mlflow_parent_run = mlflow_client.get_run(job_name)

print("Parent Run: ")
print(mlflow_parent_run)

Parent Run: 
<Run: data=<RunData: metrics={'AUC_macro': 0.9442633462069479,
 'AUC_micro': 0.9795048011808609,
 'AUC_weighted': 0.9442633462069481,
 'accuracy': 0.9152636883925098,
 'average_precision_score_macro': 0.8241834777332924,
 'average_precision_score_micro': 0.9800798378260417,
 'average_precision_score_weighted': 0.9544901293375855,
 'balanced_accuracy': 0.7645028339705318,
 'f1_score_macro': 0.7773895268653437,
 'f1_score_micro': 0.9152636883925098,
 'f1_score_weighted': 0.9131363593971006,
 'log_loss': 0.23949491741258805,
 'matthews_correlation': 0.5562323797216543,
 'norm_macro_recall': 0.5290056679410634,
 'precision_score_macro': 0.7925153276316099,
 'precision_score_micro': 0.9152636883925098,
 'precision_score_weighted': 0.9115919038485016,
 'recall_score_macro': 0.7645028339705318,
 'recall_score_micro': 0.9152636883925098,
 'recall_score_weighted': 0.9152636883925098,
 'weighted_accuracy': 0.9529157927206361}, params={}, tags={'automl_best_child_run_id': 'dynamic_fr

In [21]:
# Print parent run tags. 'automl_best_child_run_id' tag should be there.
print(mlflow_parent_run.data.tags)

{'custom tag': 'custom value', 'model_explain_run': 'best_run', 'pipeline_id_000': '5dfac790c5c209f98a1da2dc1c7fb76f0397324f;c7af0367625be6ac5c2fecbfc72ed444cb7a2111;799d2168db11fc19b9e1c6c1df62f8981ad39fe9;__AutoML_Ensemble__;__AutoML_Stack_Ensemble__', 'score_000': '0.9148970272823249;0.9118635939323221;0.7368249430460632;0.9152636883925098;0.9143636328276934', 'predicted_cost_000': '0;0;0.5;0;0', 'fit_time_000': '0.2517304;1.0225922;0.6604823999999999;3;8', 'training_percent_000': '100;100;100;100;100', 'iteration_000': '0;1;2;3;4', 'run_preprocessor_000': 'MaxAbsScaler;MaxAbsScaler;MaxAbsScaler;;', 'run_algorithm_000': 'LightGBM;XGBoostClassifier;ExtremeRandomTrees;VotingEnsemble;StackEnsemble', 'automl_best_child_run_id': 'dynamic_frog_vww91ltjw6_3', 'model_explain_best_run_child_id': 'dynamic_frog_vww91ltjw6_3', 'mlflow.rootRunId': 'dynamic_frog_vww91ltjw6', 'mlflow.runName': 'dynamic_frog_vww91ltjw6', 'mlflow.user': 'Vijay Gadhave'}


### Get the AutoML best child run

In [22]:
# Get the best model's child run

best_child_run_id = mlflow_parent_run.data.tags["automl_best_child_run_id"]
print("Found best child run id: ", best_child_run_id)

best_run = mlflow_client.get_run(best_child_run_id)

print("Best child run: ")
print(best_run)

Found best child run id:  dynamic_frog_vww91ltjw6_3
Best child run: 
<Run: data=<RunData: metrics={'AUC_macro': 0.9442633462069479,
 'AUC_micro': 0.9795048011808609,
 'AUC_weighted': 0.9442633462069481,
 'accuracy': 0.9152636883925098,
 'average_precision_score_macro': 0.8241834777332924,
 'average_precision_score_micro': 0.9800798378260417,
 'average_precision_score_weighted': 0.9544901293375855,
 'balanced_accuracy': 0.7645028339705318,
 'f1_score_macro': 0.7773895268653437,
 'f1_score_micro': 0.9152636883925098,
 'f1_score_weighted': 0.9131363593971006,
 'log_loss': 0.23949491741258805,
 'matthews_correlation': 0.5562323797216543,
 'norm_macro_recall': 0.5290056679410634,
 'precision_score_macro': 0.7925153276316099,
 'precision_score_micro': 0.9152636883925098,
 'precision_score_weighted': 0.9115919038485016,
 'recall_score_macro': 0.7645028339705318,
 'recall_score_micro': 0.9152636883925098,
 'recall_score_weighted': 0.9152636883925098,
 'weighted_accuracy': 0.9529157927206361}, 

In [23]:
# Get best model run's metrics
best_run.data.metrics

{'log_loss': 0.23949491741258805,
 'f1_score_weighted': 0.9131363593971006,
 'recall_score_macro': 0.7645028339705318,
 'balanced_accuracy': 0.7645028339705318,
 'AUC_weighted': 0.9442633462069481,
 'AUC_macro': 0.9442633462069479,
 'recall_score_weighted': 0.9152636883925098,
 'average_precision_score_macro': 0.8241834777332924,
 'precision_score_macro': 0.7925153276316099,
 'accuracy': 0.9152636883925098,
 'AUC_micro': 0.9795048011808609,
 'norm_macro_recall': 0.5290056679410634,
 'precision_score_weighted': 0.9115919038485016,
 'weighted_accuracy': 0.9529157927206361,
 'recall_score_micro': 0.9152636883925098,
 'average_precision_score_micro': 0.9800798378260417,
 'f1_score_macro': 0.7773895268653437,
 'f1_score_micro': 0.9152636883925098,
 'precision_score_micro': 0.9152636883925098,
 'average_precision_score_weighted': 0.9544901293375855,
 'matthews_correlation': 0.5562323797216543}

### Download the best model locally

In [24]:
import os

# Create local folder
local_dir = "./artifact_downloads"
if not os.path.exists(local_dir):
    os.mkdir(local_dir)

In [25]:
# Download run's artifacts/outputs

local_path = download_artifacts(run_id=best_run.info.run_id, artifact_path="outputs", dst_path=local_dir)
print("Artifacts downloaded in: {}".format(local_path))
print("Artifacts: {}".format(os.listdir(local_path)))

Artifacts downloaded in: /mnt/batch/tasks/shared/LS_root/mounts/clusters/compute-instance01/code/Users/vijaygadhave199/artifact_downloads/outputs
Artifacts: ['.amlignore', '.amlignore.amltmp', 'conda_env_v_1_0_0.yml', 'engineered_feature_names.json', 'env_dependencies.json', 'featurization_summary.json', 'generated_code', 'internal_cross_validated_models.pkl', 'mlflow-model', 'model.onnx', 'model.pkl', 'model_onnx.json', 'pipeline_graph.json', 'run_id.txt', 'scoring_file_pbi_v_1_0_0.py', 'scoring_file_v_1_0_0.py', 'scoring_file_v_2_0_0.py']


In [26]:
# Show the contents of the MLFlow model folder
os.listdir("./artifact_downloads/outputs/mlflow-model")

['.amlignore',
 '.amlignore.amltmp',
 'conda.yaml',
 'MLmodel',
 'model.pkl',
 'python_env.yaml',
 'requirements.txt']

# 7 - Create managed online endpoint

In [ ]:
# import required libraries

from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
    Environment,
    CodeConfiguration,
    ProbeSettings,
)
from azure.ai.ml.constants import ModelType

In [ ]:
# Creating unique endpoint name with current datetime to avoid conflicts
import datetime

online_endpoint_name = "bankmarketing-" + datetime.datetime.now().strftime("%m%d%H%M%f")

In [ ]:
# Create an online endpoint

endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="Online endpoint for mlflow model",
    auth_mode="key",
    tags={"foo": "bar"}
)

In [ ]:
ml_client.begin_create_or_update(endpoint).result()

# 8 - Register best model

In [ ]:
model_name = "bankmarketing-model"

model = Model(
    path=f"azureml://jobs/{best_run.info.run_id}/outputs/artifacts/outputs/mlflow-model/",
    name=model_name,
    description="classification model",
    type=AssetTypes.MLFLOW_MODEL
)

registered_model = ml_client.create_or_update(model)

In [ ]:
registered_model.id

# 9 - Deploy best model

In [ ]:
deployment = ManagedOnlineDeployment(
    name="bankmarketing-deploy",
    endpoint_name=online_endpoint_name,
    model=registered_model.id,
    instance_type="Standard_DS2_V2",
    instance_count=1,
    liveness_probe=ProbeSettings(
        failure_threshold=30,
        success_threshold=1,
        timeout=2,
        period=10,
        initial_delay=2000
    ),
    readiness_probe=ProbeSettings(
        failure_threshold=10,
        success_threshold=1,
        timeout=10,
        period=10,
        initial_delay=2000
    ),
)

In [ ]:
ml_client.online_deployments.begin_create_or_update(deployment).result()

In [ ]:
print(endpoint.traffic)

In [ ]:
endpoint.traffic = {"bankmarketing-deploy":100}
ml_client.begin_create_or_update(endpoint)

In [ ]:
print(endpoint.traffic)

# 10 - Test the deployment

In [ ]:
# Create MLTable

import mltable

paths = [{'file': './data/test-mltable-folder/bank_marketing_test_data.csv'}]

test_table = mltable.from_delimited_files(paths)
test_table.save("./data/test-mltable-folder")

In [ ]:
# test deployment with sample data

import pandas as pd

test_data = pd.read_csv("./data/test-mltable-folder/bank_marketing_test_data.csv")
test_data = test_data.drop("y", axis=1)

# Convert Data to JSON format
test_data_json = test_data.to_json(orient="records", indent=4)
data = (
    '{ \
          "input_data": {"data": '
    + test_data_json
    + "}}"
)

#  Prepare Request File
request_file_name = "sample-request-bankmarketing.json"

with open(request_file_name, "w") as request_file:
    request_file.write(data)

# Invoke the Deployed Model Endpoint
ml_client.online_endpoints.invoke(
    endpoint_name=online_endpoint_name,
    deployment_name="bankmarketing-deploy",
    request_file=request_file_name,
)

In [ ]:
# Get the details for online endpoint
endpoint = ml_client.online_endpoints.get(name=online_endpoint_name)

# existing traffic details
print(endpoint.traffic)

# Get the scoring URI
print(endpoint.scoring_uri)

In [ ]:
# Delete Deployment and endpoint

ml_client.online_endpoints.begin_delete(name=online_endpoint_name)

# 11 - Use auto generated model training code for retraining on new data

### Create Compute Target

In [45]:
# Create or Attach existing AmlCompute

from azure.ai.ml.entities import AmlCompute
from azure.core.exceptions import ResourceNotFoundError

compute_name = "cpu-cluster"

try:
    _ = ml_client.compute.get(compute_name)
    print("Found existing compute target.")
except ResourceNotFoundError:
    print("Creating a new compute target...")
    compute_config = AmlCompute(
        name=compute_name,
        type="amlcompute",
        size="STANDARD_DS3_V2",
        idle_time_before_scale_down=120,
        min_instances=0,
        max_instances=6,
    )
    ml_client.begin_create_or_update(compute_config).result()

Found existing compute target.


### Register the dataset

In [46]:
# Create MLTable

import mltable

paths = [{'file': './data/retrained-mltable-folder/bank_marketing_retrain_data.csv'}]

retrain_table = mltable.from_delimited_files(paths)
retrain_table.save('./data/retrained-mltable-folder')

In [47]:
retrain_data = Data(
    path="./data/retrained-mltable-folder/",
    type=AssetTypes.MLTABLE,
    description="Updated training dataset",
    name="retrain-bankmarketing-data"
)

retrain_data = ml_client.data.create_or_update(retrain_data)

In [48]:
import mlflow

# Obtain the tracking URI from MLClient
MLFLOW_TRACKING_URI = ml_client.workspaces.get(name=ml_client.workspace_name).mlflow_tracking_uri
print(MLFLOW_TRACKING_URI)

# Set the MLFLOW TRACKING URI
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

print("\nCurrent tracking uri: {}".format(mlflow.get_tracking_uri()))

azureml://westus.api.azureml.ms/mlflow/v1.0/subscriptions/3b57d2fe-08b1-4fe9-b535-f5c4387b9a66/resourceGroups/automl-rg/providers/Microsoft.MachineLearningServices/workspaces/automl-ws

Current tracking uri: azureml://westus.api.azureml.ms/mlflow/v1.0/subscriptions/3b57d2fe-08b1-4fe9-b535-f5c4387b9a66/resourceGroups/automl-rg/providers/Microsoft.MachineLearningServices/workspaces/automl-ws


### Download script

In [49]:
from mlflow.tracking.client import MlflowClient

# Initialize MLFlow client
mlflow_client = MlflowClient()

# Get the parent run
job_name = returned_job.name
mlflow_parent_run = mlflow_client.get_run(job_name)
print("Parent Run: ")
print(mlflow_parent_run)

# Get the best model's child run
best_child_run_id = mlflow_parent_run.data.tags["automl_best_child_run_id"]
print("Found best child run id: ", best_child_run_id)
best_run = mlflow_client.get_run(best_child_run_id)

# Download run's artifacts/outputs
local_dir = "./artifact_downloads/"
if not os.path.exists(local_dir):
    os.mkdir(local_dir)
local_path = download_artifacts(
    run_id=best_run.info.run_id, artifact_path="outputs", dst_path=local_dir
)
print("Artifacts downloaded in: {}".format(local_path))
print("Artifacts: {}".format(os.listdir(local_path)))

Parent Run: 
<Run: data=<RunData: metrics={'AUC_macro': 0.9442633462069479,
 'AUC_micro': 0.9795048011808609,
 'AUC_weighted': 0.9442633462069481,
 'accuracy': 0.9152636883925098,
 'average_precision_score_macro': 0.8241834777332924,
 'average_precision_score_micro': 0.9800798378260417,
 'average_precision_score_weighted': 0.9544901293375855,
 'balanced_accuracy': 0.7645028339705318,
 'f1_score_macro': 0.7773895268653437,
 'f1_score_micro': 0.9152636883925098,
 'f1_score_weighted': 0.9131363593971006,
 'log_loss': 0.23949491741258805,
 'matthews_correlation': 0.5562323797216543,
 'norm_macro_recall': 0.5290056679410634,
 'precision_score_macro': 0.7925153276316099,
 'precision_score_micro': 0.9152636883925098,
 'precision_score_weighted': 0.9115919038485016,
 'recall_score_macro': 0.7645028339705318,
 'recall_score_micro': 0.9152636883925098,
 'recall_score_weighted': 0.9152636883925098,
 'weighted_accuracy': 0.9529157927206361}, params={}, tags={'automl_best_child_run_id': 'dynamic_fr

In [50]:
# Find the curated environment used for training

import yaml

# constructing the path to the "MLmodel"
model_file = os.path.join(local_dir, "outputs", "mlflow-model", "MLmodel")

with open(model_file, "r") as model_stream:
    model_yaml = yaml.safe_load(model_stream)

    # extracting the name of the training environment from the model_yaml dictionary
    training_environment_name = (
        "AzureML-AutoML:" + model_yaml["metadata"]["azureml.base_image"].split(":")[-1]
    )
    print("Training emvironment {}".format(training_environment_name))

Training emvironment AzureML-AutoML:167


### Create and run command

In [51]:
# constructs a string that defines the command to be executed in the job
command_str = f"python script.py --training_dataset_uri {retrain_data.path}"

command_job = command(
    code="./artifact_downloads/outputs/generated_code/",
    command=command_str,
    tags=dict(automl_child_run_id=best_run.info.run_id),
    environment=training_environment_name,
    compute=compute_name,
    experiment_name=exp_name,
)

script_job = ml_client.create_or_update(command_job)